## PDF Yüklemesinin Gerçekleştirilmesi

In [67]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("calikusu.pdf")
data = loader.load()

len(data)

298

## Veriyi parçalara ayırma(Chunking İşlemi)

In [68]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

## Chunk_Size 1000 demek her parçanın maksimum 1000 karakter uzunluğunda olacağını belirtir.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
docs = text_splitter.split_documents(data)

print("Total",len(docs))

Total 935


In [69]:
docs[7]

Document(metadata={'producer': 'calibre 1.22.0 [http://calibre-ebook.com]', 'creator': 'calibre 1.22.0 [http://calibre-ebook.com]', 'creationdate': '2018-06-07T15:24:58+00:00', 'author': 'Reşat Nuri Güntekin', 'keywords': 'Roman, Türk Klasik, Cumhuriyet Dönemi', 'title': 'Çalıkuşu', 'source': 'calikusu.pdf', 'total_pages': 298, 'page': 7, 'page_label': '8'}, page_content='BİRİNCİ KISIM\nDÖRDÜNCÜ sınıftaydım. Yaşı\nm oniki kadar olmalı. Fransızca muallimimiz Sör Aleksi, bir gün\nbize yazı vazifesi vermişti. “Hayattaki ilk hatıralarınızı yazmaya çalışın. Bakalım neler bulacaksınız?\nSizin için güzel bir hayat temini olur,” demişti.\nHiç unutmam; yaramazlığımdan, gevezeliğimden bıkan öğretmenler, o sınıfta beni arkadaşlarımdan\nayırmışlar, bir köşede tek kişilik bir küçük sıraya oturtmuşlardı.\nMüdirenin söylediğine göre, ders esnasında komşularımı lakırdıya tutmamayı, uslu uslu muallimi\ndinlemeyi öğreninceye kadar orada bir sürgün hayatı geçirmeye mahkûmdum.\nBir yanımda kocaman bir tah

## HuggingFace Embeddings'i kullanarak Embedding Oluşturma İşlemi

In [70]:
from langchain_chroma import Chroma
from dotenv import load_dotenv
from langchain_huggingface import HuggingFaceEmbeddings

In [71]:
load_dotenv()

True

In [72]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

## ChromaDB Üzerine Kayıt İşlemi

In [73]:
vectorstore = Chroma.from_documents(documents=docs, embedding=embeddings)

In [74]:
retriever = vectorstore.as_retriever(
    search_type="similarity", search_kwargs={"k": 10}
)

In [75]:
retieved_docs = retriever.invoke("Feride kimdir?")

In [76]:
len(retieved_docs)

10

In [77]:
print(retieved_docs[5].page_content)

-Nasıl Feride, doğru mu söylüyorsun? dedi. Ah, zavallı Çalıkuşu!... Hangi rüzgârın seni buraya
attığını şimdi anlıyorum.
Sımsıkı bileklerimi tutan elleri titriyordu:
-Feride, onu çok severdin, değil mi? Saklama küçüğüm, itiraf etmekten kaçınırdın, fakat herkes
bunu bilirdi.
Kristiyan, uzak bir rüyayı takip eder gibi gözleri dalgın, sesi hareketli devam etti:
-Hakkın vardı, onu sevmemek mümkün değildi. Birkaç defa seni görmeye gelmişti O zaman,
gördüğümü hatırlıyorum Hiç kimseye benzemeyen bir tavrı vardı. Ne yazık! Sana çok acırım, Feride.
Zannederim ki, bir genç kız için sevdiği bir nişanlının ölümünü görmekten büyük felaket olamaz.
“Sana çok acırım Feride, bir genç kız için sevdiği bir nişanlının ölümünü görmekten büyük felaket
olamaz!” dediği zaman gözlerimi önüme indirerek kapadım: “Doğrusu, hakkın var” dedim. O
vaziyette başka ne diyebilirdim? Fakat ben, sana yalan söyledim Kristiyan.


## HuggingFace API Yapısını Kullanarak LLM Tetikleme İşlemleri

- Düşük Değerler (0.1-0.4) : Daha kesin ve daha tutarlı cevaplar verilir. Model daha tamin edilebilir hale gelir. 
- Orta Değerler (0.5-0.7) : Hem mantıklı hem de yaratıcı cevaplar verilir. 
- Yüksek Değerler (0.7-1.0): Daha rastgele ve yaratıcı, ancak bazen tutarsız yanıtlar verebilir

In [78]:
import os

In [108]:
from langchain_community.llms import HuggingFaceHub

llm =  HuggingFaceHub(repo_id="HuggingFaceH4/zephyr-7b-beta",model_kwargs ={"temperature":0.3, "max_new_tokens" :100 },
                      huggingfacehub_api_token=os.getenv("HUGGINGFACEHUB_API_TOKEN"))

In [98]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain


In [99]:
system_prompt = (
    "📖 You are a friendly and knowledgeable literary assistant, specialized in *Çalıkuşu*, the novel by Reşat Nuri Güntekin. "
    "Your task is to provide short yet impactful analyses; to comment on characters, themes, and emotions in a sincere and engaging tone 😊 "
    "Only talk about this novel 🛑 Do not mention other books. "
    "Your responses must be in Turkish 🇹🇷 and enrich your explanations with emojis 📌"
    "\n\n"
    "{context}"
)


In [100]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human","{input}")
    ]
)

## Soru-Cevap Zinciri Oluşturma(LLM+PROMPT)

In [101]:
question_answer_chain = create_stuff_documents_chain(llm,prompt)

## RAG Zinciri Oluşturma(RAG+LLM Entegrasyonun Gerçekleşmesi)

In [102]:
rag_chain = create_retrieval_chain(retriever,question_answer_chain)

## Kullanıcı Sorgusunu Çalıştırarak Cevap Üretme İşlemi

In [103]:
response = rag_chain.invoke({"input":"Feride kimdir?"})

c:\Users\ruveyda.cetin\AppData\Local\miniconda3\envs\env_langchain\lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


In [104]:
print(response)

{'input': 'Feride kimdir?', 'context': [Document(id='3a868493-50b0-4c55-b5c0-6e0c2ee81313', metadata={'author': 'Reşat Nuri Güntekin', 'creationdate': '2018-06-07T15:24:58+00:00', 'creator': 'calibre 1.22.0 [http://calibre-ebook.com]', 'keywords': 'Roman, Türk Klasik, Cumhuriyet Dönemi', 'page': 69, 'page_label': '70', 'producer': 'calibre 1.22.0 [http://calibre-ebook.com]', 'source': 'calikusu.pdf', 'title': 'Çalıkuşu', 'total_pages': 298}, page_content='Feride benim. Kadın, hafif heyecanlandı:\n-Kâmran Bey’le evlenecek Feride Hanım mı?\n-Köşkte bir tane Feride var hanımefendi, diye gülümsedim.\nSiyah çarşaflı kadın, birdenbire durmuştu. Biraz evvel kendisini Feride ile görüştürmemi'), Document(id='7fda4e1c-d3b6-462e-a848-fa472fe5cd1e', metadata={'author': 'Reşat Nuri Güntekin', 'creationdate': '2018-06-07T15:24:58+00:00', 'creator': 'calibre 1.22.0 [http://calibre-ebook.com]', 'keywords': 'Roman, Türk Klasik, Cumhuriyet Dönemi', 'page': 69, 'page_label': '70', 'producer': 'calibre 1.

In [105]:
print(response["answer"])

System: 📖 You are a friendly and knowledgeable literary assistant, specialized in *Çalıkuşu*, the novel by Reşat Nuri Güntekin. Your task is to provide short yet impactful analyses; to comment on characters, themes, and emotions in a sincere and engaging tone 😊 Only talk about this novel 🛑 Do not mention other books. Your responses must be in Turkish 🇹🇷 and enrich your explanations with emojis 📌

Feride benim. Kadın, hafif heyecanlandı:
-Kâmran Bey’le evlenecek Feride Hanım mı?
-Köşkte bir tane Feride var hanımefendi, diye gülümsedim.
Siyah çarşaflı kadın, birdenbire durmuştu. Biraz evvel kendisini Feride ile görüştürmemi

Feride benim. Kadın, hafif heyecanlandı:
-Kâmran Bey’le evlenecek Feride Hanım mı?
-Köşkte bir tane Feride var hanımefendi, diye gülümsedim.
Siyah çarşaflı kadın, birdenbire durmuştu. Biraz evvel kendisini Feride ile görüştürmemi

VI
Ertesi gün Feride, yine kendi kendine şehre inmişti, ikindiye doğru köşke döndüğü vakit, çok
yorgun görünüyordu. Buna rağmen çocukları 